# Le Boostrap, en pratique

Alors, j'ai vu la question passé, et effectivement, sans exemple, le sujet est pas super super clair... 
Du coup je me suis dit que ça serait sympa de faire un petit exemple expliqué pour illustrer cette méthode de création d'intervalles de confiance.

Vu qu'on a tous bossé sur le TP de stat, j'ai essayé de créer un algo de bootstrap en partant de ce dernier.

### Import des librairies

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import scipy.stats as stats
from decimal import Decimal
from sklearn import linear_model
from sklearn import metrics
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

### Chargement de la dataframe, et création de 2/3 fonctions utiles 

In [2]:
df_invest=pd.read_csv("https://bitbucket.org/portierf/shared_files/downloads/invest.txt", sep = " ")

Je vais créer deux fonctions. Une première fonction qui me permet de récupérer rapidement l'estimateur des moindres carrés (que j'appelle ici $\hat\beta$) en fonctioner de la matrice X et de Y.

La deuxième fonction me servira pour le Bootstrap, j'y reviens plus tard

In [9]:
def b_est(X,y):
    
    """Calcul l'estimateur des moindres carrés"""
    
    X_mat = np.concatenate([ones,X],axis=1)
    Gram = X_mat.transpose().dot(X_mat)
    inv_Gram = np.linalg.inv(Gram)
    return inv_Gram.dot(X_mat.transpose().dot(y))

def new_b():
    
    """Calcul l'estimateur des moindres carrées après avoir tiré aléatoirement nos données parmis nos données"""
    
    i=[random.randint(0,len(y)-1) for k in range(len(y))]
    new_X = np.matrix([np.log(df_invest.loc[k,"gnp"]) for k in i]).reshape(n,1)
    new_y = np.matrix([np.log(df_invest.loc[k,"invest"]) for k in i]).reshape(n,1)
    return b_est(new_X,new_y)

### Pourquoi le Bootstrap

Pour se donner un intervalle de confiance, on a besoin de voir à quelle distance se trouve $\hat\beta$de $\beta^*$, et on s'intéresse à la loi décrite par T = $\hat\beta$ - $\beta^*$

On a vu dans le TP qu'on pouvait calculer un intervalle pour les composantes de $\beta^*$ à l'aide de la variance de var($\hat\beta$) et de nos connaissances sur les lois normales/de student centrées réduites. Le problème est que la variance de $\hat\beta$ est directement lié à la variance de l'erreur sur Y appelé $\sigma$.

Dans le TP, on a construit un estimateur de $\sigma$ en se basant sur l'hypothèse des $\epsilon$ gaussiens iid centrés en 0.

Parfois, il n'est malheureusement pas possible d'obtenir un estimateur correct pour $\sigma$ et il n'est donc plus possible de se ramener à une loi de dispersion connue.
On va donc chercher un autre moyen d'évaluer la loi de dispersion de T = $\hat\beta$ - $\beta^*$

La théorie derrière le bootstrap 